<a href="https://colab.research.google.com/github/mjtv128/pdf_query/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q cassio datasets langchain openai pymupdf fitz tiktoken pycryptodome pikepdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 26.5 MB/s eta 0:00:00


In [ ]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

from datasets import load_dataset

import cassio

In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.4 MB/s eta 0:00:00


In [ ]:
from PyPDF2 import PdfReader
from Crypto.Cipher import AES
from pikepdf import Pdf
from urllib.request import urlopen
from io import BytesIO
import fitz

In [ ]:
# pdfreader = PdfReader('/content/NEA11_FS.pdf')
doc = fitz.open("/content/NEA11_FS.pdf")

In [ ]:
!pip uninstall pycryptodome

Found existing installation: pycryptodome 3.20.0
Uninstalling pycryptodome-3.20.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/Crypto/*
    /usr/local/lib/python3.10/dist-packages/pycryptodome-3.20.0.dist-info/*
Proceed (Y/n)? Y
  Successfully uninstalled pycryptodome-3.20.0


In [ ]:
pdfreader = Pdf.open('/content/NEA11_FS.pdf')
len(pdfreader.pages)

14

In [ ]:
len(doc)

14

In [ ]:
for i , page in enumerate(doc):
  print(page)

page 0 of /content/NEA11_FS.pdf
page 1 of /content/NEA11_FS.pdf
page 2 of /content/NEA11_FS.pdf
page 3 of /content/NEA11_FS.pdf
page 4 of /content/NEA11_FS.pdf
page 5 of /content/NEA11_FS.pdf
page 6 of /content/NEA11_FS.pdf
page 7 of /content/NEA11_FS.pdf
page 8 of /content/NEA11_FS.pdf
page 9 of /content/NEA11_FS.pdf
page 10 of /content/NEA11_FS.pdf
page 11 of /content/NEA11_FS.pdf
page 12 of /content/NEA11_FS.pdf
page 13 of /content/NEA11_FS.pdf


In [ ]:
from typing_extensions import Concatenate
raw_text = ''
for i, page in enumerate(doc):
  content = page.get_text()
  if content:
    raw_text += content

In [ ]:
raw_text

'THIRD QUARTER REPORT\nSEPTEMBER 30, 2023\nNEW ENTERPRISE ASSOCIATES 11, \nLIMITED PARTNERSHIP\nThe pages found within this report, whether or not marked "Proprietary and Confidential / Trade Secrets", contain highly\nconfidential information, including trade secret, commercial and financial information regarding New Enterprise Associates\' \ninvestment partnership (the "NEA Partnership") and the business, operations and financial condition of its respective\nportfolio companies (the "Portfolio Companies").\nThe information found within this report is for use only by our limited\npartners and shall be maintained in strict confidence. Any disclosure of this information could cause competitive harm to\nNew Enterprise Associates, the NEA Partnership, and its Portfolio Companies.\nBy reading the information found within\nthis report, each limited partner agrees that this information (i) shall be used by such limited partner solely in furtherance\nof its interests as a limited partner and s

In [ ]:
cassio.init(token= ASTRA_DB_APPLICATION_TOKEN, database_id = ASTRA_DB_ID)

ERROR:cassandra.connection:Closing connection <AsyncoreConnection(137216662549744) f8d3e868-0046-4b4f-a0bd-3aec0edda63d-us-east1.db.astra.datastax.com:29042:da4b7bf4-d0da-4d63-9400-9ed0a495c23b> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


In [ ]:
llm = OpenAI(openai_api_key = OPENAI_API_KEY)
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [ ]:
astra_vector_store = Cassandra(
    embedding = embedding,
    table_name = 'qa_mini_demo',
    session=None,
    keyspace=None,
)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator = '\n',
    chunk_size = 800,
    chunk_overlap = 200,
    length_function = len,

)

texts = text_splitter.split_text(raw_text)

In [ ]:
# texts[:50]

In [ ]:
astra_vector_store.add_texts(texts[:50])
print("Inserted %i headlines." % len(texts[:50]))
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 50 headlines.


In [ ]:
first_question = True
while True:
  if first_question:
    query_text = input('\n Enter your question or (type quit to exit):').strip()

  else:
    query_text = input('\n Whats your next questions').strip()

  if query_text.lower() == 'quit':
    break

  if query_text == '':
    continue

  first_question = False

  print("\nQUESTION: \"%s\"" % query_text)
  answer = astra_vector_index.query(query_text, llm=llm).strip()
  print("ANSWER: \"%s\"\n" % answer)

  print("FIRST DOCUMENTS BY REVELANCE")
  # for doc, score in astra_vector_store.similarity_search_with_score(query_text):
  #   print("  [%0.4f] \"%s ... \"" & (score, doc.page_content))




 Enter your question or (type quit to exit):What is the name of the fund

QUESTION: "What is the name of the fund"


ANSWER: "NEA 11 or New Enterprise Associates 11"

FIRST DOCUMENTS BY REVELANCE

 Whats your next questionsWhat is the total sum of distribution

QUESTION: "What is the total sum of distribution"


ANSWER: "The total sum of distributions is (3,561,955,613) dollars."

FIRST DOCUMENTS BY REVELANCE


KeyboardInterrupt: Interrupted by user

In [ ]:
doc.page_content

'FUND OVERVIEW \nThe table below summarizes key measures of the Partnership as of the current quarter and the most recent prior \nreporting periods.     \nTotal Value is the sum of all distributions (cash and in-kind) and the Net Assets of the Partnership.  Net Cash is the sum of all balance \nsheet accounts excluding Portfolio Investments.  Contributed Capital includes all capital contributions except deemed contributions by \nthe General Partner.  Fund Multiple is the ratio of Total Value to Contributed Capital (excluding deemed contributions by the General \nPartner).  LP Multiple is the ratio of the limited partners’ Total Value to the limited partners’ Contributed Capital.  Fund IRR and LP IRR,'